# Premade Models

Although it is possible to define your own custom model (see [custom models](custom_models.ipynb)), most use cases will want a common set of spectra with a common set of properties. To avoid unnecessarily redefining common models every time Synthesizer is used we provide some premade models which can be used "out of the box" or as the foundation for constructing more complex models.

These premade models can be imported directly from the `emission_models` submodule which also defines several lists detailing all available models.

In [ ]:
from synthesizer.emission_models import (
    AGN_MODELS,
    COMMON_MODELS,
    PREMADE_MODELS,
    STELLAR_MODELS,
)

print(STELLAR_MODELS)
print(AGN_MODELS)
print(COMMON_MODELS)
print(PREMADE_MODELS)

To use any of these models the model simply needs importing and the required arguments passed, which arguments are required depends on the exact operation, more complex models with a deeper "tree" will require more parameters to be passed to populate their "child" models deeper in the tree. This tree terminology will become clear as you read on.

In addition to the arguments specfic to each type of model, any model can be passed arguments to define a mask (the attribute to define a mask with, ``mask_attr``; the operator to use in the mask, ``mask_op``; the threshold for the mask, ``mask_thresh``). We'll also demonstrate this below but note that this is possible for all models.

As you can see in the code above, there are ``STELLAR_MODELS``, ``AGN_MODELS``, and ``COMMON_MODELS``. In the following sections we'll detail each of these.

## Stellar Emission Models

Before we detail each stellar model we'll first need a grid to pass to each model.

In [ ]:
from synthesizer.dust.attenuation import PowerLaw
from synthesizer.dust.emission import Blackbody
from synthesizer.grid import Grid
from unyt import Myr, dimensionless, kelvin

# Get the grid which we'll need for extraction
grid_dir = "../../../tests/test_grid"
grid_name = "test_grid"
grid = Grid(grid_name, grid_dir=grid_dir)

### IncidentEmission

Incident emission is an extraction.

In [ ]:
from synthesizer.emission_models import IncidentEmission

incident = IncidentEmission(grid=grid)
print(incident)

### LineContinuumEmission

...

Here we will also use a mask (for demonstration purposes) because line continuum is going to be isolated to young stars still surrounded by their birth cloud.

In [ ]:
from synthesizer.emission_models import LineContinuumEmission

line_cont = LineContinuumEmission(
    grid, fesc=0.8, mask_attr="ages", mask_op="<", mask_thresh=10 * Myr
)
print(line_cont)

### TransmittedEmission 


In [ ]:
from synthesizer.emission_models import TransmittedEmission

transmitted = TransmittedEmission(grid=grid, fesc=0.1)
print(transmitted)

### EscapedEmission

Escaped emission is the portion of the transmitted emission which fully escapes without being effected by the ISM.

In [ ]:
from synthesizer.emission_models import EscapedEmission

escaped = EscapedEmission(grid, fesc=0.1)
print(escaped)

Notice here that the "escape fraction" for the escaped model is ``1 - fesc`` because escaped is the inverse of transmitted.

### NebularContinuumEmission

...

Like ``LineContinuum`` we will also use a mask here (for demonstration purposes) because nebular continuum will be isolated to young stars still surrounded by their birth cloud.


In [ ]:
from synthesizer.emission_models import NebularContinuumEmission

nebular_cont = NebularContinuumEmission(
    grid, fesc=0.2, mask_attr="ages", mask_op="<", mask_thresh=10 * Myr
)
print(nebular_cont)

### NebularEmission

Nebular emission is the emission from a young stats birth cloud. Unlike other models we've shown so far ``NebularEmission`` comes in two forms depending on whether a lyman alpha escape fraction less than 1.0 is passed. If a lyman alpha escape fraction is passed then ``NebularEmission`` is the combination of ``LineContinuumEmission`` and ``NebularContinuum``, if not it is an extraction from the ``"nebular"`` grid.

Again, like ``LineContinuumEmission`` and ``NebularContinuum`` we'll include a mask for young stars.

In [ ]:
from synthesizer.emission_models import NebularEmission

nebular_extract = NebularEmission(grid)
print(nebular_extract)
print()

nebular_combined = NebularEmission(
    grid,
    fesc=0.1,
    fesc_ly_alpha=0.8,
    mask_attr="ages",
    mask_op="<",
    mask_thresh=10 * Myr,
)
print(nebular_combined)
nebular_combined.plot_emission_tree()

In the above we had our first model with children. For more complex models like this we can plot the "emission tree" to visualise the model. When eventually used to generate emission, the model will be traversed bottom to top (breadth first) generating each spectra.

### ReprocessedEmission 


In [ ]:
from synthesizer.emission_models import ReprocessedEmission

reprocessed = ReprocessedEmission(grid, fesc=0.1, fesc_ly_alpha=0.8)
print(reprocessed)
reprocessed.plot_emission_tree()

### EmergentEmission 


In [ ]:
from synthesizer.emission_models import EmergentEmission

emergent = EmergentEmission(
    grid=grid,
    dust_curve=PowerLaw(slope=-1),
    apply_dust_to=transmitted,
    tau_v=0.67,
    fesc=0.2,
)
print(emergent)
emergent.plot_emission_tree()

### TotalEmission



In [ ]:
from synthesizer.emission_models import TotalEmission

total = TotalEmission(
    grid=grid,
    dust_curve=PowerLaw(slope=-1),
    tau_v=0.67,
    fesc=0.2,
    fesc_ly_alpha=0.7,
    dust_emission_model=Blackbody(temperature=100 * kelvin),
)
print(total)
total.plot_emission_tree()

This is the most complex model we've seen so far (but you'll see below that this is still quite simple). However, this is a good point to stop and review the meaning of different features in the emission tree.

In the emission tree plot we can see each individual spectra generated by the model and how they relate. Solid lines denote combinations of spectra into another, dashed lines denote an attenuation of the child spectra to produce the parent, blue boxes have no mask applied, green boxes (which we currently have none of) have a mask applied to them, rounded boxes are the result of a non-extraction operation, and square boxes are an extraction operation. You'll notice the extraction operations are always leaves in the tree.

Note that omitting `dust_emission_model` when initialising `TotalEmission` will result in a simpler model where `total == emergent`.

### CharlotFall2000

In addition to the simpler models shown above we also implement some more complex models. 

The first of these follows Charlot&Fall+2000 including emission split into young and old populations (with the threshold defined by ``age_pivot``) with dust attenuation specific to each population.

In [ ]:
from synthesizer.emission_models import CharlotFall2000

cf_model = CharlotFall2000(
    grid=grid,
    tau_v_ism=1.0,
    tau_v_nebular=0.7,
    dust_curve_ism=PowerLaw(slope=-0.7),
    dust_curve_nebular=PowerLaw(slope=-1.3),
    age_pivot=7 * dimensionless,
    dust_emission_ism=Blackbody(temperature=50 * kelvin),
    dust_emission_nebular=Blackbody(temperature=100 * kelvin),
)
print(cf_model)
cf_model.plot_emission_tree(fontsize=6)

If we omit the dust emission model then we get a simpler model with a root at ``"emergent"``.

In [ ]:
cf_model = CharlotFall2000(
    grid=grid,
    tau_v_ism=1.0,
    tau_v_nebular=0.7,
    dust_curve_ism=PowerLaw(slope=-0.7),
    dust_curve_nebular=PowerLaw(slope=-1.3),
    age_pivot=7 * dimensionless,
)
cf_model.plot_emission_tree()

It's also possible to plot a subtree within a model by passing the root of the subtree. This is particularly helpful for models like `CharlotFall2000` which define a lot of extra spectra that don't necessarily appear in the main tree depending on the exact inputs. You can see this in the plots below which show the trees defining the "extra" spectra available when using ``CharlotFall2000`` (and indeed ``PacmanEmission`` and ``BimodelPacmanEmission`` which we'll cover shortly).

In [ ]:
cf_model.plot_emission_tree(root="incident")

In [ ]:
cf_model.plot_emission_tree(root="transmitted")

In [ ]:
cf_model.plot_emission_tree(root="intrinsic")

In [ ]:
cf_model.plot_emission_tree(root="attenuated")

### PacmanEmission

We also implement a generalised form of Charlot&Fall+2000 which accounts for escape fractions called "Pacman". The Pacman model has two forms, the first is the ``PacmanEmission`` model which unlike ``CharlotFall2000`` does not differentiate between young and old populations.

In [ ]:
from synthesizer.emission_models import PacmanEmission

# Simple Pacman with dust emission
simple_pc_model = PacmanEmission(
    grid=grid,
    tau_v=0.7,
    dust_curve=PowerLaw(slope=-1.3),
    dust_emission=Blackbody(temperature=100 * kelvin),
    fesc=0.2,
    fesc_ly_alpha=0.9,
)
simple_pc_model.plot_emission_tree()

Again, omitting the ``dust_emission_model`` results in a model with ``"emergent"`` at it's root.

In [ ]:
simple_pc_model = PacmanEmission(
    grid=grid,
    tau_v=0.7,
    dust_curve=PowerLaw(slope=-1.3),
    fesc=0.2,
    fesc_ly_alpha=0.9,
)
simple_pc_model.plot_emission_tree()

### BiomodalPacmanEmission

The second flavour of the Pacman model is the ``BimodalPacmanEmisison`` model which is the same as ``PacmanEmission`` in every other respect but does differentiate between old and young populations.

In [ ]:
from synthesizer.emission_models import BimodalPacmanEmission

pc_model = BimodalPacmanEmission(
    grid=grid,
    tau_v_ism=1.0,
    tau_v_nebular=0.7,
    dust_curve_ism=PowerLaw(slope=-1.3),
    dust_curve_nebular=PowerLaw(slope=-0.7),
    dust_emission_ism=Blackbody(temperature=100 * kelvin),
    dust_emission_nebular=Blackbody(temperature=30 * kelvin),
    fesc=0.2,
    fesc_ly_alpha=0.9,
)
pc_model.plot_emission_tree(fontsize=5)

And as with all other models, the dust emission can omitted.

In [ ]:
pc_model = BimodalPacmanEmission(
    grid=grid,
    tau_v_ism=1.0,
    tau_v_nebular=0.7,
    dust_curve_ism=PowerLaw(slope=-1.3),
    dust_curve_nebular=PowerLaw(slope=-0.7),
    fesc=0.2,
    fesc_ly_alpha=0.9,
)
pc_model.plot_emission_tree(fontsize=6)

## AGN Emission Models

In addition to stellar models you can also make models for black holes. You are, of course, free to construct whatever emission model you want using the preexisting AGN grids but we define a set of premade models including a complex `UnifiedAGN` model to calculate spectra for an AGN's disc, NLR, BLR, and torus.

In [ ]:
from synthesizer import Grid
from synthesizer.dust.emission import Blackbody
from synthesizer.emission_models import UnifiedAGN
from unyt import kelvin

# Get the NLR and BLR grids
nlr_grid = Grid("test_grid_agn-nlr", grid_dir="../../../tests/test_grid")
blr_grid = Grid("test_grid_agn-blr", grid_dir="../../../tests/test_grid")

model = UnifiedAGN(
    nlr_grid,
    blr_grid,
    covering_fraction_nlr=0.1,
    covering_fraction_blr=0.1,
    torus_emission_model=Blackbody(1000 * kelvin),
)
print(model)

model.plot_emission_tree()